In [2]:
!optimum-cli export onnx --model 'nvidia/NV-Embed-v2' nv-embed-v2-openvino

Traceback (most recent call last):
  File "/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/site-packages/optimum/exporters/onnx/__main__.py", line 258, in main_export
    task = TasksManager.infer_task_from_model(model_name_or_path)
  File "/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/site-packages/optimum/exporters/tasks.py", line 1747, in infer_task_from_model
    inferred_task_name = cls._infer_task_from_model_name_or_path(
  File "/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/site-packages/optimum/exporters/tasks.py", line 1710, in _infer_task_from_model_name_or_path
    raise KeyError(f"Could not find the proper task name for the model {model_name_or_path}.")
KeyError: 'Could not find the proper task name for the model nvidia/NV-Embed-v2.'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jy006/miniconda3/envs/paper-parser/bin/optimum-cli", line 8, in <module>
    sys.exit(main())

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

model_name = "nvidia/NV-Embed-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

NVEmbedModel(
  (latent_attention_model): LatentAttentionModel(
    (cross_attend_blocks): ModuleList(
      (0): PreNorm(
        (fn): Attention(
          (to_q): Linear(in_features=4096, out_features=32768, bias=False)
          (to_kv): Linear(in_features=4096, out_features=65536, bias=False)
          (to_out): Linear(in_features=32768, out_features=4096, bias=False)
        )
        (norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (norm_context): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      )
      (1): PreNorm(
        (fn): FeedForward(
          (net): Sequential(
            (0): Linear(in_features=4096, out_features=32768, bias=True)
            (1): GEGLU()
            (2): Linear(in_features=16384, out_features=4096, bias=True)
          )
        )
        (norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (embedding_model): BidirectionalMistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (la

In [3]:
import deepspeed

# 减少显存占用，可以使用半精度
# 如果显卡支持 bfloat16 也可以改成 torch.bfloat16
dtype = torch.float16
model = model.half()

# mp_size=1 表示不做张量并行，适合单卡或仅想用DeepSpeed优化kernel的情况
# 如果有多张GPU并想做模型并行，可将 mp_size 设为 GPU 数量。
engine = deepspeed.init_inference(
    model=model,
    mp_size=1,
    replace_method="auto",        # DeepSpeed会自动替换支持的层
    replace_with_kernel=True      # 使用已优化的自定义kernel
)

ModuleNotFoundError: No module named 'deepspeed'

In [11]:
with torch.no_grad():
    outputs = engine.module.encode([
        "PD-L1 expression in cancer patients receiving anti PD-1\/PD-L1 antibodies: A systematic review and meta-analysis\nBackground: Despite the success of immunotherapy directed at inhibiting of programmed death-1 (PD-1)\/PD-ligand (L)1 signaling, it is not established whether PD-L1 expression correlates with the clinical response and outcome in different tumors. The present meta-analysis investigates whether the PD-L1 status, detected by immunohistochemistry, is associated with clinical response and mortality in patients treated with anti-PD-1\/PD-L1 therapy. Methods: A systematic literature search and quantitative analysis were planned, conducted and reported following CONSORT and QUORUM checklists, up to December 2015, to identify clinical trials with information on cancer outcome by PD-L1 immunohistochemical expression in tumor tissues. We used random effects models to estimate Summary Objective Response Rates (SORRs) and Summary Odd Ratio (SOR) for the comparison of PD-L1 positive and negative patients. Results: We summarized 20 trials carried out in metastatic melanoma (MM), non-small cell lung cancer (NSCLC), and renal cell carcinoma (RCC) patients receiving anti-PD-1\/PD-L1 antibodies (4230 MM, 1417 NSCLC and 312 RCC patients). Positive PD-L1 MM patients showed a significant decrease (53%) in the risk of mortality vs. negative cases with no heterogeneity. Furthermore, SORRs were 45% and 27% in PD-L1 positive and negative patients, respectively, and SOR indicates a significant difference in term of responses: 2.14 (95% CI: 1.65, 2.77), with low between-study heterogeneity (I 2 = 35%). Furthermore, results from randomized clinical trials on MM showed that PD-L1 expression is significantly associated with greater clinical response rates to anti-PD1 treatments (SOR 1.89; 95%CI: 1.35, 2.64) but not to other treatments (SOR 0.96; 95%CI: 0.5, 1.87). In non-squamous NSCLC SORRs were 29% and 11% in PD-L1 positive and negative patients, respectively, and SOR indicates a significant difference between responses: 3.78 (1.54, 9.24), with no between-study heterogeneity. Squamous NSCLC and RCC did not show any significant difference in response according to the PD-L1 status. Conclusion: PD-L1 expression is significantly associated with mortality and clinical response to anti-PD-1\/PD-L1 antibodies in MM patients and with clinical response in patients with non-squamous NSCLC."
    ], instruction="", max_length=32768)

/Users/jy006/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v2/5130cf1daf847c1bacee854a6ef1ca939e747fb2/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


tensor([[ 0.0071, -0.0291,  0.0052,  ..., -0.0105,  0.0044,  0.0002]])